<a href="https://colab.research.google.com/github/Jeniejean/ML/blob/main/TISCO_Stock_Price_Prediction_6610422019_Nutthida_Yotaprasert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas
!pip install numpy
!pip install requests
!pip install lxml
!pip install yfinance

In [4]:
import yfinance as yf
mbk = yf.Ticker('TISCO.BK')

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()
